## Setup

First, let's install the required packages and set up our API key.

In [ ]:
# Install required packages (if needed)
# !pip install google-generativeai pandas numpy pillow opencv-python gradio python-dotenv

In [ ]:
# Set up environment
import sys
import os

# Add parent directory to path
sys.path.append(os.path.dirname(os.getcwd()))

# Set your Gemini API key
os.environ['GEMINI_API_KEY'] = 'your_api_key_here'  # Replace with your actual API key

print("✓ Environment configured")

## Import ChefByte Agent

In [ ]:
from agent.orchestrator import ChefByteAgent
from agent.tools.vision_tool import extract_ingredients_from_image
from agent.tools.receipt_ocr_tool import extract_items_from_receipt
from agent.tools.ingredient_normalizer import normalize_ingredients
from agent.tools.recipe_search import search_recipes
from agent.tools.nutrition_estimator import calculate_nutrition

print("✓ ChefByte modules loaded")

## Demo 1: Test Individual Tools

Let's test each tool separately to understand how they work.

### 1.1 Vision Tool - Extract Ingredients from Image

In [ ]:
# Test with a fridge image (replace with your image path)
# ingredients = extract_ingredients_from_image('path/to/your/fridge.jpg')

# For demo, let's use mock data
ingredients = [
    {"name": "chicken breast", "quantity": "2 pieces", "condition": "fresh"},
    {"name": "tomatoes", "quantity": "5", "condition": "fresh"},
    {"name": "broccoli", "quantity": "1 head", "condition": "good"},
    {"name": "rice", "quantity": "1 bag", "condition": "good"},
    {"name": "cheddar cheese", "quantity": "1 block", "condition": "good"}
]

print("Ingredients detected:")
for ing in ingredients:
    print(f"  • {ing['name']} ({ing['quantity']}) - {ing['condition']}")

### 1.2 Ingredient Normalizer

In [ ]:
# Normalize ingredient names
normalized = normalize_ingredients(ingredients)

print("Normalized ingredients:")
for ing in normalized:
    print(f"  • {ing}")

### 1.3 Recipe Search

In [ ]:
# Search for recipes
recipes = search_recipes(
    normalized,
    dietary_constraints=['gluten-free'],
    max_missing=2
)

print(f"Found {len(recipes)} matching recipes:\n")
for i, recipe in enumerate(recipes[:5], 1):
    print(f"{i}. {recipe['name']}")
    print(f"   Match: {recipe['match_percentage']}")
    print(f"   Missing: {', '.join(recipe['missing_ingredients']) if recipe['missing_ingredients'] else 'none'}")
    print()

### 1.4 Nutrition Calculator

In [ ]:
# Calculate nutrition
nutrition_data = calculate_nutrition(
    recipes,
    calorie_target=2000,
    meal_count=3
)

print("Selected Meals:\n")
for i, meal in enumerate(nutrition_data['selected_meals'], 1):
    print(f"{i}. {meal['name']}")
    print(f"   Calories: {meal['nutrition']['calories']}")
    print(f"   Protein: {meal['nutrition']['protein_g']}g")
    print(f"   Carbs: {meal['nutrition']['carbs_g']}g")
    print(f"   Fat: {meal['nutrition']['fat_g']}g")
    print()

print("\nTotal Daily Nutrition:")
for key, value in nutrition_data['total_nutrition'].items():
    print(f"  {key}: {value}")

## Demo 2: Full Agent Workflow

Now let's run the complete ChefByte agent.

In [ ]:
# Initialize the agent
agent = ChefByteAgent()

print("✓ ChefByte agent initialized")

In [ ]:
# Run the agent
# Replace with your actual image paths if available
result = agent.run(
    fridge_image=None,  # Or: 'path/to/fridge.jpg'
    receipt_image=None,  # Or: 'path/to/receipt.jpg'
    dietary_constraints=['vegetarian', 'high-protein'],
    calorie_target=2000,
    meal_count=3
)

print("\n" + "="*60)
print("MEAL PLAN GENERATED")
print("="*60)
print(result['meal_plan'])

## Demo 3: View Agent Memory

Let's inspect what the agent remembers.

In [ ]:
# Get memory state
memory_state = agent.get_memory_state()

print("Agent Memory State:")
print("="*40)
for key, value in memory_state.items():
    print(f"{key}: {value}")

## Demo 4: Custom Meal Plan

Create a custom meal plan with specific requirements.

In [ ]:
# Reset agent for new session
agent.reset()

# Create custom meal plan
custom_result = agent.run(
    fridge_image=None,
    dietary_constraints=['vegan', 'gluten-free'],
    calorie_target=1800,
    meal_count=4
)

print("\n" + "="*60)
print("CUSTOM MEAL PLAN")
print("="*60)
print(custom_result['meal_plan'])

## Demo 5: Analyze Results

Let's visualize the meal plan data.

In [ ]:
import pandas as pd

# Create a DataFrame from the results
if 'selected_meals' in nutrition_data:
    meals_df = pd.DataFrame([
        {
            'Meal': meal['name'],
            'Calories': meal['nutrition']['calories'],
            'Protein (g)': meal['nutrition']['protein_g'],
            'Carbs (g)': meal['nutrition']['carbs_g'],
            'Fat (g)': meal['nutrition']['fat_g']
        }
        for meal in nutrition_data['selected_meals']
    ])
    
    print("\nMeal Plan Summary:")
    print(meals_df.to_string(index=False))

## Next Steps

1. **Try with real images**: Upload your own fridge photos and receipts
2. **Customize constraints**: Experiment with different dietary requirements
3. **Add recipes**: Expand the recipes.csv database
4. **Improve nutrition data**: Update nutrition.csv with more ingredients
5. **Launch the UI**: Run `python ui/gradio_ui.py` for the web interface

## Resources

- [Google Gemini API Docs](https://ai.google.dev/docs)
- [Gradio Documentation](https://www.gradio.app/docs)
- [ChefByte GitHub](https://github.com/your-repo/ChefByte)